# Preference Alignment with Direct Preference Optimization (DPO)

In Questo notebook andremo a svolgere il processo di fine-tuning di un LLM utilizzando Direct Preference Optimization (DPO). Useremo il modello **SmolLM-135M-Instruct** il qule è già affinato con il SFT Training, quindi è compatibile per fare il DPO fine-tuning. Possiamo anche utilizzare il nostro modello affinato con SFT nel primo modulo **./01_Instruction_Tuning/SmolLM2-FT-MyDataset**

In [1]:
from huggingface_hub import login

login()

## Carichiamo il modello affinato con SFT su HuggingFace

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "../01_Instruction_Tuning/SmolLM2-FT-MyDataset"
repo_name = "felipe93/smolLM2-ft"  # ✅ Usa il tuo username corretto

# Carica tokenizer e modello
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Carica su Hugging Face Hub
print("📤 Caricamento tokenizer...")
tokenizer.push_to_hub(repo_name)
print("📤 Caricamento modello...")
model.push_to_hub(repo_name)

print(f"✅ Modello caricato con successo su https://huggingface.co/{repo_name}")


📤 Caricamento tokenizer...
📤 Caricamento modello...


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

✅ Modello caricato con successo su https://huggingface.co/felipe93/smolLM2-ft


# Ora carichiamo il modello dalla repo creata 

In [8]:
tokenizer = AutoTokenizer.from_pretrained("felipe93/smolLM2-ft")
model = AutoModelForCausalLM.from_pretrained("felipe93/smolLM2-ft")
print(model)

tokenizer_config.json:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/139 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb)

## Impotiamo le librerie

In [9]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Carichiamo il dataset
Usiamo il dataset [ultrafeedback_binarized](https://huggingface.co/datasets/trl-lib/ultrafeedback_binarized)

In [10]:
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")
print(dataset)

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 62135
})


In [14]:
print(dataset[0]['chosen'])

[{'content': 'Use the pygame library to write a version of the classic game Snake, with a unique twist', 'role': 'user'}, {'content': "Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.\n5. Once we have these objects set up, we can sta

In [15]:
print(dataset[0]['rejected'])

[{'content': 'Use the pygame library to write a version of the classic game Snake, with a unique twist', 'role': 'user'}, {'content': 'Sure, here\'s an example of how to write a version of Snake game with a unique twist using the Pygame library:\n```python\nimport pygame\n\nclass SnakeGame:\n    def __init__(self, game_width, game_height):\n        pygame.init()\n        screen = pygame.display.set_mode((game_width, game_height))\n        pygame.display.set_caption("Snake Game")\n        self.speed = 5  # Speed of the snake\n        self.food_speed = 1  # Speed of the food\n        self.direction = 0  # Initial direction of the snake\n        self.snakelen = 0  # Length of the snake\n        self.food = pygame.image.load("snake_food.png")\n        self.head = pygame.image.load("snake_head.png")\n        self.tail = pygame.image.load("snake_tail.png")\n        self.game Quint()\n    def Quint(self):\n        for i in range(50):\n            pygame.draw.line(screen, (180, 100, 220), (0, 

Come vediamo il dataset ha due colonne **chosen** e **rejected** dove per la stessa richiesta vi sono le due risposte una corretta e l'altra no, come il modello non deve rispondere.

## Selezioniamo il nostro modello

Usiamo il nostro modello affinato con SFT partendo dal modello base **SmolLM-135M**, il quale è già adatto per DPO.

In [16]:
model_name = "felipe93/SmolLM2-ft"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)


model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32
).to(device)
# ogni passo (token generato non usa K-V dell'attenzione degli stati precedenti) viene ricalcolato da zero
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
# token di padding (di fine sequenza)
tokenizer.pad_token = tokenizer.eos_token

# Impostiamo il notro nome per il modello affinato 
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_1"]

## Addestriamo il modello con DPO

In [17]:
# Training Arguments
training_args = DPOConfig(
    #training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy='no',
    logging_steps=1,
    output_dir="smol_dpo_output",
    warmup_steps=100,
    bf16=True,
    report_to='none',
    remove_unused_columns=False,
    hub_model_id=finetune_name,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536
)

## ⚙️ PARAMETRI BASE DI TRAINING

### ✅ `per_device_train_batch_size=4`

> Numero di esempi (prompt + chosen + rejected) elaborati in parallelo **su ogni GPU o dispositivo**.

* Se hai 1 GPU, è la batch size effettiva per ogni step.
* L'**effective batch size totale** viene calcolato insieme a `gradient_accumulation_steps`.

---

### ✅ `gradient_accumulation_steps=4`

> Quanti step **accumulare i gradienti prima** di eseguire l’ottimizzazione.

* In pratica, fa finta di avere `4 × 4 = 16` batch size totale, senza saturare la memoria GPU.
* Consente di addestrare con **batch virtuali più grandi**, utile con modelli più grandi o GPU piccole.

---

### ✅ `gradient_checkpointing=True`

> Abilita il salvataggio di memoria **ricostruendo gli attivatori** durante il backward pass.

* ✅ Pro: Riduce molto il consumo di RAM/VRAM
* ❌ Contro: Rallenta un po’ il training
* Molto utile su modelli da 1B+ parametri o su hardware con meno di 24 GB VRAM

---

### ✅ `learning_rate=5e-5`

> Tasso base di apprendimento, ovvero **quanto il modello cambia** per ogni passo di ottimizzazione.

* 5e-5 è una scelta abbastanza "sicura" per modelli piccoli o SFT/DPO leggeri
* Si combina con il learning rate scheduler

---

### ✅ `lr_scheduler_type="cosine"`

> Il learning rate **diminuisce seguendo una curva coseno**, che lo porta a zero in modo graduale.

* Particolarmente utile per il fine-tuning: **inizia alto, poi si stabilizza, poi si spegne dolcemente**

---

### ✅ `max_steps=200`

> Numero massimo di step di training (update steps).

* **Non sono epoch**: ogni step è un batch ottimizzato.
* Se usi un dataset piccolo o bassa batch size, può bastare anche < 1000 step.

---

### ✅ `save_strategy="no"`

> Disabilita completamente il **salvataggio automatico dei checkpoint** durante il training.

* Più leggero ma rischioso se il training si interrompe.
* Puoi aggiungere manualmente `trainer.save_model()` a fine training.

---

### ✅ `logging_steps=1`

> Ogni quanti step vengono **loggate le metriche** (loss, lr, ecc.).

* `1` = logga ad ogni step, utile per debug o esperimenti brevi.
* In produzione o per esperimenti lunghi si usa spesso `10`, `50`, `100`.

---

### ✅ `output_dir="smol_dpo_output"`

> Directory in cui **verrà salvato il modello finale** dopo il training.

* Contiene anche tokenizer e config aggiornati.
* Se esiste già, verrà sovrascritta a meno che tu non usi `resume_from_checkpoint`.

---

### ✅ `warmup_steps=100`

> I primi 100 step di training usano **learning rate crescente da 0 a `learning_rate`**.

* Stabilizza l’ottimizzazione all'inizio.
* Consigliato per DPO e fine-tuning delicati.

---

### ✅ `bf16=True`

> Usa la **precisione bfloat16** (se supportata da hardware).

* Richiede GPU **Ampere+** (es. A100, RTX 30xx).
* Risparmia VRAM e accelera il training senza perdita di qualità.

---

### ✅ `report_to="none"`

> Disabilita la loggatura su strumenti esterni (come `wandb`, `tensorboard` ecc.)

* Evita errori se non hai configurato nulla.
* Se vuoi usare `wandb`, metti `"wandb"` e imposta `WANDB_API_KEY`.

---

### ✅ `remove_unused_columns=False`

> Mantiene **tutti i campi nel dataset**, anche quelli non usati nel forward.

* ⚠️ `True` potrebbe eliminare campi come `rejected`, `chosen` o `prompt`.
* `False` è **necessario** per DPO, perché servono più campi rispetto a SFT.

---

### ✅ `use_mps_device=device == "mps"`

> Abilita l’uso del backend **Metal Performance Shaders** (solo per Mac con Apple Silicon).

* Se `device == "mps"` allora abilita ottimizzazioni per GPU Apple.
* Non ha effetto su CUDA/CPU.

---

### ✅ `hub_model_id=finetune_name`

> Specifica il nome del modello su Hugging Face Hub da usare durante l'upload automatico.

* Esempio: `"felipe93/smolLM2-dpo"`
* Verrà usato da `.push_to_hub()` o `trainer.push_to_hub()` se chiamato dopo il training.

---

## 🔥 PARAMETRI SPECIFICI DI DPO

### ✅ `beta=0.1`

> Temperatura che **modula la loss DPO**: quanto il modello dovrebbe preferire la risposta “chosen” rispetto alla “rejected”.

* Valori bassi (0.05–0.1) = **modello più cauto**, preferisce con più certezza solo output chiaramente superiori
* Valori alti (0.5–1.0) = modello più “aggressivo”, cambia facilmente

---

### ✅ `max_prompt_length=1024`

> Lunghezza massima del prompt (input dell’utente) in token.

* Serve per troncamento e padding.
* Evita crash su modelli piccoli (come `SmolLM-135M`) che non supportano prompt lunghi.

---

### ✅ `max_length=1536`

> Lunghezza massima totale (prompt + output generato) in token.

* DPO valuta l’intero output generato rispetto al prompt.
* Deve essere < `model.config.max_position_embeddings` (tipicamente 2048 o 4096).

---

## 🧠 In sintesi

Questo blocco di configurazione è ottimizzato per:

✅ Allenare un **modello Instruct di piccole dimensioni** con DPO
✅ Su hardware limitato (es. 1 GPU, M1/M2 Mac)
✅ Con preferenza per velocità, leggerezza e portabilità


## 🧠 Cos'è `gradient_checkpointing`?

In un training classico, **durante il forward pass** (cioè quando il modello elabora l’input), vengono salvati **tutti i tensori intermedi** (attivazioni) per poterli usare poi nel **backward pass** (cioè quando si calcolano i gradienti per fare l’ottimizzazione).

Questi tensori possono occupare **molta memoria**, soprattutto con modelli grandi o prompt lunghi.

---

## ✅ Con `gradient_checkpointing=True`

👉 Il modello **NON salva le attivazioni intermedie durante il forward**.

📌 Invece, **le ricalcola dinamicamente durante il backward**, al momento in cui servono per calcolare i gradienti.

> È come dire: "non salvo tutto subito, ma quando mi serve, lo ricalcolo".

---

## 📉 Differenza visiva

|             | 🧠 Normal Training             | 🧠 Gradient Checkpointing     |
| ----------- | ------------------------------ | ----------------------------- |
| Memoria GPU | Alta (tutti i tensori salvati) | Bassa (salvati solo pochi)    |
| Velocità    | Veloce                         | Più lento (perché ricalcola)  |
| Adatto a    | GPU grandi o modelli piccoli   | GPU limitate o modelli grandi |

---

## 📈 Esempio numerico

Immagina di addestrare un modello con 1.3B parametri su una GPU da 12 GB:

* 🔁 Senza `gradient_checkpointing`: crasha per mancanza di memoria
* ✅ Con `gradient_checkpointing=True`: funziona, ma impiega 20–30% di tempo in più

---

## ⚠️ Compatibilità e attenzione

* Funziona con quasi tutti i modelli **Transformer-based** (es. GPT, BERT, Falcon, LLaMA, ecc.)
* Alcune librerie (es. `DeepSpeed`, `accelerate`, `bitsandbytes`) lo richiedono o lo abilitano di default
* Può interferire con `use_cache=True` → spesso si disattiva `use_cache` quando lo si usa

---

## ✅ Quando usarlo?

| Caso                                                 | Lo uso?                        |
| ---------------------------------------------------- | ------------------------------ |
| Modello < 500M su GPU da 24 GB                       | ❌ No                           |
| Modello da 1B+ o prompt lunghi su 12–16 GB VRAM      | ✅ Sì                           |
| Addestramento su Apple M1/M2 (8–16 GB RAM condivisa) | ✅ Sì                           |
| Deployment/inference                                 | ❌ No (rallenta senza benefici) |

---

## 🔁 In pratica

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("felipe93/smolLM2-ft")
model.gradient_checkpointing_enable()
```

Oppure:

```python
training_args = TrainingArguments(
    ...
    gradient_checkpointing=True,
    ...
)
```

---

## 🧠 In sintesi

| ✅ Vantaggi                     | ❌ Svantaggi                       |
| ------------------------------ | --------------------------------- |
| Usa molta meno memoria GPU     | Più lento (ricalcola attivazioni) |
| Permette batch più grandi      | Debug più difficile               |
| Essenziale su hardware modesto | Aumenta il tempo per ogni step    |




## 🧠 Cos'è `beta` in DPO?

Nel DPO, il modello è addestrato su **coppie di risposte**:

* `prompt` → una **risposta preferita** (`chosen`)
* `prompt` → una **risposta rifiutata** (`rejected`)

L’obiettivo è far sì che il modello assegni **una probabilità più alta alla risposta preferita**, rispetto alla rifiutata.

---

## 📐 Formula semplificata della loss DPO

```text
L = -log(σ(β × (log p(chosen) - log p(rejected))))
```

Dove:

* `σ` è la **funzione sigmoide**
* `log p(chosen)` è il log-likelihood della risposta preferita
* `log p(rejected)` è quello della risposta rifiutata
* **`β` è il moltiplicatore della differenza**, ovvero **quanto peso dare alla preferenza**

---

## 🔧 Cosa fa `beta` in pratica?

* Se `β` è **grande** → anche piccole differenze tra `chosen` e `rejected` causano **grandi aggiornamenti**
* Se `β` è **piccolo** → il modello sarà **più cauto**, cambia solo se la differenza è chiara

---

## 🎛️ Effetto dei valori di `beta`

| `beta` | Effetto sul modello                               | Quando usarlo                        |
| ------ | ------------------------------------------------- | ------------------------------------ |
| `1.0`  | Forte spinta a preferire le risposte `chosen`     | Quando il dataset è molto pulito     |
| `0.5`  | Moderata sensibilità                              | Default bilanciato                   |
| `0.1`  | Comportamento **cauto**, solo preferenze evidenti | ✅ Consigliato per SFT + DPO          |
| `0.01` | Quasi nessuna influenza (molto conservativo)      | Per test, debugging o low-signal DPO |

---

## 🧪 Esempio intuitivo

Immagina queste log-probabilità:

```text
log p(chosen)   = -1.2
log p(rejected) = -1.5
```

Differenza = 0.3

### Con β=1.0:

```text
σ(1.0 × (0.3)) ≈ 0.57 → Loss ≈ -log(0.57) ≈ 0.56
```

### Con β=0.1:

```text
σ(0.1 × (0.3)) ≈ 0.51 → Loss ≈ -log(0.51) ≈ 0.67
```

➡️ Il **gradiente sarà più piccolo** → aggiornamenti più morbidi

---

## ✅ Quale valore usare?

* Per modelli piccoli o dataset rumorosi → **`beta=0.1`** (come nel tuo caso)
* Per dataset molto curati (es. `HH-RLHF`) o DPO finale su modello robusto → `0.5` o `1.0`
* In fase di debug o esperimenti → prova anche `0.01` per vedere se il modello sta apprendendo

---

## 📌 In sintesi

| Aspetto     | `beta` controlla...                                           |
| ----------- | ------------------------------------------------------------- |
| Intensità   | Quanto fortemente il modello segue le preferenze              |
| Sensibilità | A che punto una differenza è considerata rilevante            |
| Stabilità   | Valori più bassi rendono il training più stabile              |
| Overfitting | Valori alti possono portare a overfit se i dati sono rumorosi |

---


In [19]:
chat_template = """{% for message in messages %}{{ '<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>\\n' }}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\\n' }}{% endif %}"""

tokenizer.chat_template = chat_template
tokenizer.save_pretrained("../01_Instruction_Tuning/SmolLM2-FT-MyDataset")  # oppure altra cartella tokenizer


('../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\tokenizer_config.json',
 '../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\special_tokens_map.json',
 '../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\vocab.json',
 '../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\merges.txt',
 '../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\added_tokens.json',
 '../01_Instruction_Tuning/SmolLM2-FT-MyDataset\\tokenizer.json')

In [20]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    # beta=0.1,
    # Maximum length of the input prompt in tokens
    # max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    # max_length=1536,
)

Applying chat template to train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

In [21]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

c:\Users\felip\Desktop\smol-course\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\felip\Desktop\smol-course\.venv\lib\site-packages\transformers\trainer.py:3700: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  ctx_manager = torch.cpu.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Step,Training Loss
1,0.693100
2,0.693100
3,0.693200
4,0.709100
5,0.716900
6,0.691900
7,0.686800
8,0.699100
9,0.681300
10,0.670200
